# Preprocessamento

In [1]:
attr = ['fiocruz_id', 'Título', 'Resumo', 'Palavras-chaves do autor', 'Veículo de publicação']

In [2]:
cnpq = ['new_cnpq_level_1a',
        'new_cnpq_level_2a',
        'new_cnpq_level_3a',
        'new_cnpq_level_4a']

In [3]:
all = attr + cnpq

## Lendo a base

In [4]:
import pandas as pd

In [5]:
df = pd.read_excel('unidades_26082024.xlsx', dtype=str, na_filter=False)

In [6]:
# substitui campos com {} por ''
df = df.replace('{}', '')
# remove caracteres como {, " } dos campos
df = df.replace('[\\{\\}"]', '', regex=True)
# remove escape de linha, inclusive literal
df.replace(to_replace=[r'\\t|\\n|\\r', '\t|\n|\r'], value=['',''], regex=True, inplace=True)
# seleciona somente publicações com pelo menos grande área e área
df = df.loc[(df['new_cnpq_level_1a'] != '') & (df['new_cnpq_level_2a'] != '')]
# removendo amostras da ENSP
df = df[df['Unidade de origem do rótulo '] != 'Fiocruz/Escola Nacional de Saúde Pública Sérgio Arouca']
# removendo amostras com titulos duplicados
df = df.drop_duplicates(subset=['Título'])
# removendo amostras sem resumo e palavras-chave
df = df[df['Resumo'] != '']
df = df[df['Palavras-chaves do autor'] != '']
# selecionando os atributos de interesse
df = df[all]

In [7]:
df.shape

(8155, 9)

## Traduzindo para o Inglês

In [8]:
from tqdm import tqdm
from multiprocessing import Pool
from deep_translator import exceptions, GoogleTranslator

In [9]:
tqdm.pandas()

In [10]:
translator = GoogleTranslator(source='auto', target='en')

In [11]:
def chunkstring(text, length=4995):
    return [text[0+i:length+i] for i in range(0, len(text), length)]
        
def translate(text, translator=translator):
    try:
        return ''.join(translator.translate(substring) for substring in chunkstring(text))
    except exceptions.TooManyRequests as t:
        raise
    return text

In [12]:
def concat_translate(row):
    text = row['Título'] + ' ' + row['Resumo'] + ' ' +  row['Palavras-chaves do autor'] + ' ' + row['Veículo de publicação']
    return translate(text)

In [13]:
df['all'] = df.progress_apply(concat_translate, axis=1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8155/8155 [1:06:12<00:00,  2.05it/s]


## Gravando a base traduzida

In [14]:
df.to_csv('unidades_26082024-translated.csv')